#  파이토치를 활용한 딥러닝 구현 흐름  
1. 데이터 전처리, 딥러닝 모델의 입력과 출력 확인  
2. 데이터셋 클래스 작성  
3. 데이터 로더 활용  
4. 딥러닝 모델 작성  
5. 순전파 정의  
6. 손실함수 정의  
7. 최적화 기법 설정
8. 하이퍼파라미터 설정
9. 학습/검증 시행  
10. 테스트 데이터로 추론  

## 3.1 선형회귀

In [143]:
import torch
import torch.nn as nn # nn 모듈에는 뉴럴 네트워크를 구성하기 위해 필요한 모든 요소가 구현되어 있다. ex) Linear, Conv, RNN, 활성화함수 등
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init

In [141]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


## 옵티마이저 

In [163]:
optimizer = optim.SGD(linear_model.parameters(), lr=0.001)
optimizer = optim.Adam(linear_model.parameters(), lr=0.001)
#optimizer = optim.RMSprop(...)

## 손실함수

In [164]:
# 퀴즈 MSELoss와 크로스엔트로피 로스는 각각 어떤 문제에 사용할까요?
mse_loss = nn.MSELoss()
ce_loss = nn.CrossEntropyLoss()
bce_loss = nn.BCELoss()

In [165]:
# 손실함수를 사용해서 직접 계산해봅시다.
y_hat = torch.tensor([1.1, 2.4, 3.1])
y = torch.tensor([1, 2, 3])
loss1 = mse_loss(y_hat, y)
print(loss1)

tensor(0.0600)


In [182]:
torch.manual_seed(1)
output = torch.rand([2, 10])
target = torch.LongTensor([1, 9])

print(output)
print(output.shape)
print(target.shape)
print(ce_loss(output, target))

tensor([[0.7576, 0.2793, 0.4031, 0.7347, 0.0293, 0.7999, 0.3971, 0.7544, 0.5695,
         0.4388],
        [0.6387, 0.5247, 0.6826, 0.3051, 0.4635, 0.4550, 0.5725, 0.4980, 0.9371,
         0.6556]])
torch.Size([2, 10])
torch.Size([2])
tensor(2.4004)


In [167]:
torch.manual_seed(1)
sigmoid = nn.Sigmoid()
loss = nn.BCELoss()
input_ = torch.randn(3, requires_grad=True)
print(input_)
print(input_.shape)

target = torch.empty(3).random_(2)
print(target)
print(target.shape)
output = loss(sigmoid(input_), target)
output2 = loss(sigmoid(torch.tensor(9.2)), torch.tensor(1.0))
output3 = loss(sigmoid(torch.tensor(-9.2)), torch.tensor(1.0))
print(output)
print(output2)
print(output3)
# 3개의 독립적인 이진 분류 문제로 접근, 
# Binary Cross Entropy 함수를 여러 개의 클래스를 가지는 문제의loss 계산에 활용할 수 있음.
# 참고 : https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-7/06-multi_classification

tensor([0.6614, 0.2669, 0.0617], requires_grad=True)
torch.Size([3])
tensor([1., 0., 0.])
torch.Size([3])
tensor(0.6587, grad_fn=<BinaryCrossEntropyBackward0>)
tensor(0.0001)
tensor(9.2001)


In [168]:
y = torch.tensor([0.35, -2.1, -0.28])
y_hat = sigmoid(y)
target = torch.tensor([1.0, 0.0, 1.0])
loss2 = bce_loss(y_hat, target)
print(loss2)

tensor(0.4973)


## 선형회귀

In [ ]:
# 간단한 선형 회귀 모델  y_pred = wx + b
linear_model = nn.Linear(1, 1)
label = y_noise

In [177]:
# 노이즈가 추가된 데이터 생성
num_data = 1000
num_epoch = 100
x = init.uniform_(torch.Tensor(num_data, 1), -10, 10)
noise = init.normal_(torch.FloatTensor(num_data, 1), std=1)
y = 2*x + 3
y_noise = y + noise

optimizer = optim.SGD(linear_model.parameters(), lr=0.01)


In [178]:
## print(linear_model.weight)
print(linear_model.weight)
print(linear_model.bias)
print(linear_model.weight.grad)
print(linear_model.bias.grad)

Parameter containing:
tensor([[2.0012]], requires_grad=True)
Parameter containing:
tensor([2.5398], requires_grad=True)
tensor([[0.0007]])
tensor([-0.9456])


## 지도학습 모델의 학습 순서  
1) 옵티마이저의 그래디언트를 0으로 만든다.  
2) 데이터를 모델에 넣어서 값을 예측한다.  
3) 정답 데이터와 예측 값을 통해 손실함수를 계산한다.  
4) 자동미분 함수인 .backward()를 통해 그래디언트를 계산한다.  
5) 옵티마이저의 step()함수를 호출한다.  

In [179]:
# 학습
for i in range(num_epoch):
    optimizer.zero_grad() # 옵티마이저의 그래디언트를 0으로 초기화
    output = linear_model(x) # 모델을 통해 값을 예측
    loss = mse_loss(output, label) # 정답 데이터와 예측 값과의 차이를 통해 손실함수 계산
    print(loss.item())
    loss.backward() # 자동미분을 통해 가중치에 대한 그래디언트 계산 calculate gradient
    optimizer.step() # 계산한 그래디언트를 통해 가중치 업데이트 w = w - alpha * gradient

    if i % 10 == 0:
        print(loss.data)

275.2430725097656
tensor(275.2431)
150.18048095703125
136.97288513183594
135.57144165039062
135.41636657714844
135.39312744140625
135.38406372070312
135.37680053710938
135.36990356445312
135.36334228515625
135.35702514648438
tensor(135.3570)
135.35096740722656
135.34515380859375
135.33956909179688
135.33416748046875
135.32901000976562
135.32406616210938
135.31930541992188
135.31472778320312
135.3103485107422
135.30612182617188
tensor(135.3061)
135.3020782470703
135.2981719970703
135.29443359375
135.2908477783203
135.2873992919922
135.28408813476562
135.28089904785156
135.27784729003906
135.27491760253906
135.27207946777344
tensor(135.2721)
135.26939392089844
135.26678466796875
135.2642822265625
135.2618865966797
135.25958251953125
135.25735473632812
135.25521850585938
135.25320434570312
135.25123596191406
135.2493438720703
tensor(135.2493)
135.24752807617188
135.2458038330078
135.24412536621094
135.24253845214844
135.24098205566406
135.239501953125
135.2380828857422
135.23672485351562


In [180]:
print(linear_model.weight)
print(linear_model.bias)

Parameter containing:
tensor([[-0.0297]], requires_grad=True)
Parameter containing:
tensor([2.9756], requires_grad=True)


# 퀴즈 (Easy)  
위 모델은 y = 2x + 3 으로 학습되지 않았습니다.  
이를 개선하기 위해서는 하이퍼 파라미터를 조정해야 합니다.  
1) 위 선형회귀모델에서 하이퍼파라미터의 종류는 뭐가 있을까요?  
2) 하이퍼파라미터를 조정해서 모델의 가중치와 편향이 정답에 가깝도록 학습시켜보세요

## 3.2 다중선형회귀

In [197]:
num_data = 1000
num_epoch = 2000

x = init.uniform_(torch.Tensor(10, 3), -10, 10)
noise = init.normal_(torch.FloatTensor(10, 1), std=1)
weights = torch.tensor([2., 3., 1.])
print(x.matmul(weights).shape)
y = x.matmul(weights) + -1
y = y.unsqueeze(1)
print(f"y.shape {y.shape}")
y_noise = y + noise

torch.Size([10])
y.shape torch.Size([10, 1])


In [198]:
# 다중회귀 모델 구현
multi_model = nn.Linear(3, 1)
loss_func = nn.MSELoss()

# 옵티마이저
optimizer = optim.SGD(multi_model.parameters(), lr=0.001)
label = y_noise

In [199]:
# 학습
for i in range(num_epoch):
    optimizer.zero_grad() # 옵티마이저의 그래디언트를 0으로 초기화
    output = multi_model(x) # 모델을 통해 값을 예측
    loss = loss_func(output, label) # 정답 데이터와 예측 값과의 차이를 통해 손실함수 계산
    print(loss.item())
    loss.backward() # 자동미분을 통해 가중치에 대한 그래디언트 계산 calculate gradient
    optimizer.step() # 계산한 그래디언트를 통해 가중치 업데이트 w = w - alpha * gradient

    if i % 100 == 0:
        print(loss.data)

481.73858642578125
tensor(481.7386)
399.367431640625
332.70709228515625
278.6700134277344
234.78012084960938
199.0516357421875
169.89163208007812
146.02218627929688
126.41786193847656
110.2555160522461
96.87440490722656
85.7437515258789
76.43717956542969
68.61190032958984
61.99226760864258
56.3564453125
51.525718688964844
47.3560791015625
43.731285095214844
40.55742645263672
37.75852966308594
35.273006439208984
33.050865173339844
31.05143165588379
29.241497039794922
27.593917846679688
26.08638572692871
24.700531005859375
23.421165466308594
22.235654830932617
21.13344383239746
20.105655670166016
19.144817352294922
18.244544982910156
17.399370193481445
16.604595184326172
15.85614013671875
15.150433540344238
14.4843111038208
13.854985237121582
13.259961128234863
12.697000503540039
12.16407585144043
11.659339904785156
11.181109428405762
10.72783088684082
10.29808235168457
9.890525817871094
9.503948211669922
9.137191772460938
8.789196014404297
8.458942413330078
8.145503044128418
7.847989559

1.9580812454223633
1.9580169916152954
1.9579532146453857
1.9578883647918701
1.9578230381011963
1.9577586650848389
1.9576942920684814
1.9576303958892822
1.9575679302215576
1.9575035572052002
1.9574426412582397
1.9573795795440674
1.9573160409927368
1.9572551250457764
1.9571924209594727
1.957131028175354
1.957067847251892
1.9570081233978271
1.9569451808929443
1.9568850994110107
1.9568243026733398
1.956762671470642
1.9567010402679443
1.9566423892974854
1.9565823078155518
1.9565197229385376
1.956463098526001
1.956403374671936
1.9563424587249756
1.956284523010254
1.9562263488769531
1.9561659097671509
1.9561083316802979
1.956051230430603
1.9559917449951172
1.9559348821640015
1.955875039100647
1.9558184146881104
1.9557621479034424
1.9557034969329834
1.9556477069854736
1.9555914402008057
1.9555336236953735
1.955476999282837
1.9554214477539062
1.9553651809692383
1.9553098678588867
1.9552549123764038
1.9551969766616821
1.9551414251327515
1.9550882577896118
1.9550330638885498
1.95497727394104
1.95

1.9395592212677002
1.9395548105239868
1.939550757408142
1.9395473003387451
1.9395431280136108
1.939538598060608
1.9395360946655273
1.939531683921814
1.9395296573638916
1.9395265579223633
1.9395214319229126
1.9395160675048828
1.93951416015625
1.9395115375518799
1.9395078420639038
tensor(1.9395)
1.939504861831665
1.9395023584365845
1.9394960403442383
1.9394943714141846
1.9394893646240234
1.9394862651824951
1.93948233127594
1.9394794702529907
1.9394747018814087
1.9394710063934326
1.9394677877426147
1.939467430114746
1.9394614696502686
1.9394569396972656
1.9394559860229492
1.9394519329071045
1.9394464492797852
1.9394433498382568
1.939440131187439
1.9394385814666748
1.9394363164901733
1.9394296407699585
1.9394289255142212
1.9394232034683228
1.9394185543060303
1.939415693283081
1.9394134283065796
1.939410924911499
1.9394071102142334
1.9394031763076782
1.939400315284729
1.9393999576568604
1.9393962621688843
1.9393926858901978
1.9393885135650635
1.9393848180770874
1.9393808841705322
1.93938004

In [200]:
print(multi_model.weight)
print(multi_model.bias)

Parameter containing:
tensor([[1.8472, 2.8568, 1.1032]], requires_grad=True)
Parameter containing:
tensor([-0.7917], requires_grad=True)


# 퀴즈 (Easy)  
1) 하이퍼파라미터를 조정해서 모델의 가중치와 편향이 정답에 가깝도록 학습시켜보세요

## 3.3 로지스틱 회귀

In [55]:
# 칠판에 그리기
x_data = [[0, 2], [1, 2], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [56]:
linear_model = nn.Linear(2, 1)
sigmoid = nn.Sigmoid()

In [59]:
# 옵티마이저
optimizer = optim.SGD(linear_model.parameters(), lr=0.001)

In [60]:
# 학습
for i in range(num_epoch):
    optimizer.zero_grad() 
    output = sigmoid(linear_model(x_train))
    loss = bce_loss(output, y_train) 
    print(loss.item())
    loss.backward() 
    optimizer.step()
    if i % 10 == 0:
        print(loss.data)

1.2100101709365845
tensor(1.2100)
1.205311894416809
1.2006360292434692
1.1959822177886963
1.1913506984710693
1.1867414712905884
1.1821547746658325
1.1775903701782227
1.173048496246338
1.1685291528701782
1.1640325784683228
tensor(1.1640)
1.1595584154129028
1.1551071405410767
1.1506785154342651
1.1462726593017578
1.1418895721435547
1.1375294923782349
1.1331921815872192
1.1288777589797974
1.1245864629745483
1.120318055152893
tensor(1.1203)
1.1160727739334106
1.111850619316101
1.1076513528823853
1.1034750938415527
1.0993221998214722
1.0951924324035645
1.0910857915878296
1.0870023965835571
1.082942008972168
1.0789051055908203
tensor(1.0789)
1.074891209602356
1.070900559425354
1.0669333934783936
1.0629892349243164
1.0590683221817017
1.0551707744598389
1.051296353340149
1.047445297241211
1.0436173677444458
1.039812684059143
tensor(1.0398)
1.0360313653945923
1.0322731733322144
1.0285382270812988
1.0248264074325562
1.0211378335952759
1.017472267150879
1.0138299465179443
1.0102107524871826
1.006

In [62]:
linear_model.weight

Parameter containing:
tensor([[-0.1904,  0.2681]], requires_grad=True)

## 3.4 클래스를 통한 회귀 모델 구현

In [122]:
class LinearRegression(nn.Module):
    def __init__(self):
        # 부모클래스의 nn.Module 의 생성자를 먼저 호출한다.
        super(LinearRegression, self).__init__()
        self.linear_layer = nn.Linear(1, 1)
        
    def forward(self, x):
        # forward 메소드는 클래스 내에서 레이어들 간의 연산을 구현하는 부분이다.
        return self.linear_layer(x)

class MultiRegression(nn.Module):
    def __init__(self):
        super(MultiRegression, self).__init__()
        self.multi_layer = nn.Linear(3, 1)
    
    def forward(self, x):
        return self.multi_layer(x)

class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear_layer = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.linear_layer(x)
        x = sigmoid(x)
        return x

linear_model = LinearRegression()
multi_model = MultiRegression()
logistic_model = LogisticRegression()


## 퀴즈 (Normal)  
위 세 가지 모델을 구현했으면 또 반복문을 통해 학습시켜야 합니다.  
이는 귀찮은 과정이니 함수 형태로 만들어서 코드의 반복을 줄여봅시다.  
텐서플로우에서 사용했던 fit 함수를 직접 만들어봅시다.  
fit() 함수는 model, optimizer, loss_func, x_train, y_train, epochs를 입력으로 받습니다.  
위에서 수행한 반복문을 함수형태로 만들어서 세 가지 회귀모델에 적용할 것입니다.  

In [ ]:
def fit():
    pass

In [123]:
def fit(model, optimizer, loss_func, x_train, y_train, epochs):
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(x_train)
        L = loss_func(output, y_train) 
        L.backward() 
        optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch: {epoch + 1}/{epochs} | Loss: {L.item():4f}')


In [124]:
# dataset for linear regression
num_data = 1000
epochs = 1000

x = init.uniform_(torch.Tensor(num_data, 1), -10, 10)
noise = init.normal_(torch.FloatTensor(num_data, 1), std=1)
y = 2*x + 3
y_noise = y + noise
lr = 0.01
optimizer = optim.Adam(linear_model.parameters(), lr)
fit(linear_model, optimizer, mse_loss, x, y_noise, epochs)
print(linear_model.linear_layer.weight)

Epoch: 1/1000 | Loss: 206.707535
Epoch: 101/1000 | Loss: 80.300575
Epoch: 201/1000 | Loss: 25.145985
Epoch: 301/1000 | Loss: 6.747341
Epoch: 401/1000 | Loss: 2.115411
Epoch: 501/1000 | Loss: 1.192140
Epoch: 601/1000 | Loss: 1.023997
Epoch: 701/1000 | Loss: 0.991115
Epoch: 801/1000 | Loss: 0.984252
Epoch: 901/1000 | Loss: 0.982901
Parameter containing:
tensor([[1.9979]], requires_grad=True)


In [125]:
# dataset for multivariate regression
num_data = 1000
num_epoch = 1000
x = init.uniform_(torch.Tensor(10, 3), -10, 10)
noise = init.normal_(torch.FloatTensor(10, 1), std=1)
weights = torch.tensor([2., 3., 1.])
print(x.matmul(weights).shape)
y = x.matmul(weights) + -1
y = y.unsqueeze(1)
print(f"y.shape {y.shape}")
y_noise = y + noise
lr = 0.01
optimizer = optim.Adam(multi_model.parameters(), lr)
fit(multi_model, optimizer, mse_loss, x, y_noise, num_epoch)
print(multi_model.multi_layer.weight)

torch.Size([10])
y.shape torch.Size([10, 1])
Epoch: 1/1000 | Loss: 377.118988
Epoch: 101/1000 | Loss: 146.060760
Epoch: 201/1000 | Loss: 50.614040
Epoch: 301/1000 | Loss: 17.787920
Epoch: 401/1000 | Loss: 7.917123
Epoch: 501/1000 | Loss: 4.465247
Epoch: 601/1000 | Loss: 2.749716
Epoch: 701/1000 | Loss: 1.731630
Epoch: 801/1000 | Loss: 1.125065
Epoch: 901/1000 | Loss: 0.780732
Parameter containing:
tensor([[1.9109, 2.9347, 1.0136]], requires_grad=True)


In [131]:
# dataset for logistic regression
num_epoch = 1000
x_data = [[0, 2], [1, 2], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
lr = 0.001
optimizer = optim.Adam(logistic_model.parameters(), lr)
fit(logistic_model, optimizer, bce_loss, x_train, y_train, num_epoch)
print(logistic_model.linear_layer.weight)

Epoch: 1/1000 | Loss: 1.180484
Epoch: 101/1000 | Loss: 0.895627
Epoch: 201/1000 | Loss: 0.716588
Epoch: 301/1000 | Loss: 0.624826
Epoch: 401/1000 | Loss: 0.579649
Epoch: 501/1000 | Loss: 0.552558
Epoch: 601/1000 | Loss: 0.531382
Epoch: 701/1000 | Loss: 0.512291
Epoch: 801/1000 | Loss: 0.494334
Epoch: 901/1000 | Loss: 0.477328
Parameter containing:
tensor([[ 0.3856, -0.0820]], requires_grad=True)


## Softmax 회귀  
https://wikidocs.net/60575